# Register TSV Data With Athena
This will create an Athena table in the Glue Catalog (Hive Metastore).

Now that we have a database, we’re ready to create a table that’s based on the `Amazon Customer Reviews Dataset`. We define the columns that map to the data, specify how the data is delimited, and provide the location in Amazon S3 for the file(s). 


<img src="img/athena_register_tsv.png" width="60%" align="left">

In [1]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

In [2]:
ingest_create_athena_table_tsv_passed = False

In [3]:
%store -r ingest_create_athena_db_passed

In [4]:
try:
    ingest_create_athena_db_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [5]:
print(ingest_create_athena_db_passed)

True


In [6]:
if not ingest_create_athena_db_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')          

[OK]


In [7]:
%store -r s3_private_path_tsv

In [8]:
try:
    s3_private_path_tsv
except NameError:
    print('*****************************************************************************')
    print('[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************')
    print('[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************')
    print('*****************************************************************************')

In [9]:
print(s3_private_path_tsv)

s3://sagemaker-us-west-2-085964654406/amazon-reviews-pds/tsv


# Import PyAthena

In [10]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
from pyathena.util import as_pandas

# Create Athena Table from Local TSV Files

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [11]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = 's3://{0}/athena/staging'.format(bucket)

In [12]:
# Set Athena parameters
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'

In [13]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         marketplace string,
         customer_id string,
         review_id string,
         product_id string,
         product_parent string,
         product_title string,
         product_category string,
         star_rating int,
         helpful_votes int,
         total_votes int,
         vine string,
         verified_purchase string,
         review_headline string,
         review_body string,
         review_date string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(database_name, table_name_tsv, s3_private_path_tsv)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.amazon_reviews_tsv(
         marketplace string,
         customer_id string,
         review_id string,
         product_id string,
         product_parent string,
         product_title string,
         product_category string,
         star_rating int,
         helpful_votes int,
         total_votes int,
         vine string,
         verified_purchase string,
         review_headline string,
         review_body string,
         review_date string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-west-2-085964654406/amazon-reviews-pds/tsv'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [14]:
cursor = connect(region_name=region, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

# Verify The Table Has Been Created Succesfully

In [15]:
statement = 'SHOW TABLES in {}'.format(database_name)
cursor.execute(statement)

df_show = as_pandas(cursor)
df_show.head(5)

,tab_name
0,amazon_reviews_tsv


In [16]:
if table_name_tsv in df_show.values:
    ingest_create_athena_table_tsv_passed = True

In [17]:
%store ingest_create_athena_table_tsv_passed

Stored 'ingest_create_athena_table_tsv_passed' (bool)


# Run A Sample Query

In [18]:
product_category = 'Digital_Software'

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(database_name, table_name_tsv, product_category)

print(statement)

SELECT * FROM dsoaws.amazon_reviews_tsv
    WHERE product_category = 'Digital_Software' LIMIT 100


In [19]:
cursor = connect(region_name=region, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [20]:
df = as_pandas(cursor)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,17747349,R2EI7QLPK4LF7U,B00U7LCE6A,106182406,CCleaner Free [Download],Digital_Software,4,0,0,N,Y,Four Stars,So far so good,2015-08-31
1,US,10956619,R1W5OMFK1Q3I3O,B00HRJMOM4,162269768,ResumeMaker Professional Deluxe 18,Digital_Software,3,0,0,N,Y,Three Stars,Needs a little more work.....,2015-08-31
2,US,13132245,RPZWSYWRP92GI,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],Digital_Software,1,1,2,N,Y,One Star,Please cancel.,2015-08-31
3,US,35717248,R2WQWM04XHD9US,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,Digital_Software,5,0,0,N,Y,Works as Expected!,Works as Expected!,2015-08-31
4,US,17710652,R1WSPK2RA2PDEF,B00FZ0FK0U,574904556,SecureAnywhere Intermet Security Complete 5 De...,Digital_Software,4,1,2,N,Y,Great antivirus. Worthless customer support,I've had Webroot for a few years. It expired a...,2015-08-31


In [21]:
if not df.empty:
    print('[OK]')
else:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++')

[OK]


# Review the New Athena Table in the Glue Catalog

In [22]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(region)))


# Store Variables for the Next Notebooks

In [23]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                  -> 'automl-dm-26-16-00-25'
autopilot_endpoint_name                           -> 'automl-dm-ep-26-16-21-49'
autopilot_train_s3_uri                            -> 's3://sagemaker-us-west-2-085964654406/data/amazon
ingest_create_athena_db_passed                    -> True
ingest_create_athena_table_tsv_passed             -> True
s3_private_path_tsv                               -> 's3://sagemaker-us-west-2-085964654406/amazon-revi
s3_public_path_tsv                                -> 's3://amazon-reviews-pds/tsv'
setup_dependencies_passed                         -> True
setup_iam_roles_passed                            -> True
setup_instance_check_passed                       -> True
setup_s3_bucket_passed                            -> True


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();